# Day 2: Password Philosophy

Your flight departs in a few days from the coastal airport; the easiest way down to the coast from here is via toboggan.

The shopkeeper at the North Pole Toboggan Rental Shop is having a bad day. "Something's wrong with our computers; we can't log in!" You ask if you can take a look.

Their password database seems to be a little corrupted: some of the passwords wouldn't have been allowed by the Official Toboggan Corporate Policy that was in effect when they were chosen.

To try to debug the problem, they have created a list (your puzzle input) of passwords (according to the corrupted database) and the corporate policy when that password was set.

## Puzzle 1

For example, suppose you have the following list:

```text
1-3 a: abcde
1-3 b: cdefg
2-9 c: ccccccccc
```

Each line gives the password policy and then the password. The password policy indicates the lowest and highest number of times a given letter must appear for the password to be valid. For example, 1-3 a means that the password must contain a at least 1 time and at most 3 times.

In the above example, 2 passwords are valid. The middle password, cdefg, is not; it contains no instances of b, but needs at least 1. The first and third passwords are valid: they contain one a or nine c, both within the limits of their respective policies.

How many passwords are valid according to their policies?

In [1]:
# Python imports
from dataclasses import dataclass
from pathlib import Path
from typing import Callable

We could split the task into functions that (1) parse each password/policy combination into a data structure (e.g. dictionary or tuple) and (2) check the data structure. But the problem is small enough that we might as well write a single function that does both.

In [2]:
def validate_password(pwd):
    """Return True if the combination of password and policy is valid.
    
    :param pwd:  password and policy in <int1>-<int2> <char>: <password> format
    
    To be valid, the <password> must contain <char> at least <int1> times, and
    no more than <int2> times.
    """
    # Split password into elements
    elems = pwd.strip().split()
    minc, maxc = [int(_) for _ in elems[0].split("-")]
    char = elems[1][0]
    passwd = elems[-1]
    
    # Check password
    count = passwd.count(char)
    if count < minc:
        return False
    if count > maxc:
        return False
    
    return True

We write a function to handle the local file of password and policy data.

In [3]:
def check_password_file(fpath):
    """Load password file and return number of valid/invalid passwords
    
    :param fpath:  path to password file
    """
    valid, invalid = 0, 0
    
    with Path(fpath).open("r") as ifh:
        for line in ifh.readlines():
            if validate_password(line):
                valid += 1
            else:
                invalid += 1
                
    return (valid, invalid)

Solving the example puzzle.

In [4]:
check_password_file("day02_test.txt")

(2, 1)

Solving the real puzzle.

In [5]:
check_password_file("day02_data.txt")

(418, 582)

## Puzzle 2

While it appears you validated the passwords correctly, they don't seem to be what the Official Toboggan Corporate Authentication System is expecting.

The shopkeeper suddenly realizes that he just accidentally explained the password policy rules from his old job at the sled rental place down the street! The Official Toboggan Corporate Policy actually works a little differently.

Each policy actually describes two positions in the password, where 1 means the first character, 2 means the second character, and so on. (Be careful; Toboggan Corporate Policies have no concept of "index zero"!) Exactly one of these positions must contain the given letter. Other occurrences of the letter are irrelevant for the purposes of policy enforcement.

Given the same example list from above:

    1-3 a: abcde is valid: position 1 contains a and position 3 does not.
    1-3 b: cdefg is invalid: neither position 1 nor position 3 contains b.
    2-9 c: ccccccccc is invalid: both position 2 and position 9 contain c.

How many passwords are valid according to the new interpretation of the policies?

We don't need to touch the `check_password_file()` function - we only need to modify the `validate_password()` function to change the password policy rule.

In [6]:
def validate_password(pwd):
    """Return True if the combination of password and policy is valid.
    
    :param pwd:  password and policy in <int1>-<int2> <char>: <password> format
    
    To be valid, the <password> must contain <char> at position <int1> or <int2>.
    """
    # Split password into elements
    elems = pwd.strip().split()
    minc, maxc = [int(_) for _ in elems[0].split("-")]
    char = elems[1][0]
    passwd = elems[-1]
    
    # Check password
    if (passwd[minc-1] + passwd[maxc-1]).count(char) == 1:
        return True
    
    return False

Solving the example puzzle.

In [7]:
check_password_file("day02_test.txt")

(1, 2)

Solving the real puzzle.

In [8]:
check_password_file("day02_data.txt")

(616, 384)

## Refactoring

Although we wrote `validate_password()` as a single function in both cases to split up the password into elements, and then check the policy, the need to accommodate changes to the policy suggests we can *refactor* our solution to make future changes easier to manage.

As it stands, we have to completely replace the `validate_password()` function every time the policy changes. It would be more maintainable if we were to split up the policy check from the password parsing, like we do below. Note that we can pass *functions* to other functions, just as if they were data.

There are now more functions, and a dataclass, which looks more complicated (and it *is* more complicated than it needs to be for this puzzle), but if we needed to implement a number of different password policies, this would be more flexible and maintainable.

A change here is that we define a `dataclass` to represent the password elements. These are relatively new introductions to Python.

In [9]:
@dataclass
class PassPolicy:
    """Represents a password and password policy."""
    minc: int
    maxc: int
    char: str
    passwd: str

        
def parse_password(pwd):
    """Return password and policy as PassPolicy object.
    
    :param pwd:  password and policy in <int1>-<int2> <char>: <password> format
    
    To be valid, the <password> must contain <char> at position <int1> or <int2>.
    """
    # Split password into elements
    elems = pwd.strip().split()
    
    # Create and return PassPolicy object
    minc, maxc = [int(_) for _ in elems[0].split("-")]
    char = elems[1][0]
    passwd = elems[-1]
    
    return PassPolicy(minc, maxc, char, passwd)


def check_policy_old(pwd):
    """Return True if PassPolicy passes old policy"""
    # Check password
    count = pwd.passwd.count(pwd.char)
    if count < pwd.minc:
        return False
    if count > pwd.maxc:
        return False
    
    return True


def check_policy_new(pwd):
    """Return True if PassPolicy passes new policy"""
    # Check password
    if (pwd.passwd[pwd.minc-1] + pwd.passwd[pwd.maxc-1]).count(pwd.char) == 1:
        return True
    
    return False


def check_password_file(fpath, polcheck=check_policy_new):
    """Load password file and return count of valid/invalid passwords
    
    :param fpath:  path to password file
    :param polcheck:  function returning True if password is valid, False otherwise
    
    By default the new policy check is used, but we can substitute
    other policy check functions.
    """
    valid, invalid = 0, 0
    
    with Path(fpath).open("r") as ifh:
        for line in ifh.readlines():
            pwd = parse_password(line)
            if polcheck(pwd):
                valid += 1
            else:
                invalid += 1
                
    return (valid, invalid)

Checking first with the new policy:

In [10]:
check_password_file("day02_test.txt")

(1, 2)

In [11]:
check_password_file("day02_data.txt")

(616, 384)

Now trying the old policy:

In [12]:
check_password_file("day02_test.txt", check_policy_old)

(2, 1)

In [13]:
check_password_file("day02_data.txt", check_policy_old)

(418, 582)